### Installing dependencies
- ```db-dtypes```: needed for ```to_dataframe()``` function

In [16]:
# !pip install google-cloud-bigquery
# !pip install db-dtypes
# !pip3 install db-dtypes

In [17]:
import re
import os
import warnings
import pandas as pd
import numpy as np
from google.cloud import bigquery as bq
from google.oauth2 import service_account

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'explore29-33756158108f.json'
warnings.filterwarnings('ignore')

### Establish connections

In [18]:
client = bq.Client()

### Functions to clean numerical and datetime data

In [19]:
def clean_numeric(data):
	if data is None:
		return data
	try:
		match = re.search(r"[-+]?\d*\.?\d+", data)
		return (float(match.group()))
	except (ValueError, AttributeError, TypeError):
		return data

def clean_datetime(date_series):
	if pd.isna(date_series):
		return None
	else:
		date_series = pd.to_datetime(date_series, format='mixed', yearfirst=True, errors='coerce')
		return date_series

### Get ```customer_orders``` data

In [20]:
get_customer_data = """SELECT * FROM explore29.pizza_runner.customer_orders;"""
query = client.query(get_customer_data)
customer_orders = query.to_dataframe()

### Cleaning ```customer_orders```

In [21]:
display(
    customer_orders.head(),
    customer_orders.tail()
)

,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,101,1,,,2020-01-01 18:05:02+00:00
1,2,101,1,,,2020-01-01 19:00:52+00:00
2,6,101,2,null,null,2020-01-08 21:03:13+00:00
3,3,102,2,,null,2020-01-02 23:51:23+00:00
4,3,102,1,,,2020-01-02 23:51:23+00:00


,order_id,customer_id,pizza_id,exclusions,extras,order_time
9,9,103,1,4,"1, 5",2020-01-10 11:22:59+00:00
10,5,104,1,null,1,2020-01-08 21:00:29+00:00
11,10,104,1,"2, 6","1, 4",2020-01-11 18:34:49+00:00
12,10,104,1,null,null,2020-01-11 18:34:49+00:00
13,7,105,2,null,1,2020-01-08 21:20:29+00:00


In [22]:
customer_orders['exclusions'] = customer_orders['exclusions'].str.strip().replace('', None).replace('null', None)
customer_orders['extras'] = customer_orders['extras'].str.strip().replace('', None).replace('null', None)
customer_orders['order_time'] = customer_orders['order_time'].apply(clean_datetime)

customer_orders.tail()

,order_id,customer_id,pizza_id,exclusions,extras,order_time
9,9,103,1,4,"1, 5",2020-01-10 11:22:59+00:00
10,5,104,1,None,1,2020-01-08 21:00:29+00:00
11,10,104,1,"2, 6","1, 4",2020-01-11 18:34:49+00:00
12,10,104,1,None,None,2020-01-11 18:34:49+00:00
13,7,105,2,None,1,2020-01-08 21:20:29+00:00


### Update Bigquery dataset

In [23]:
destination_table = 'explore29.pizza_runner.std_customer_orders'

job_config = bq.LoadJobConfig(
	write_disposition='WRITE_TRUNCATE'  # overwrite the table if it exists
)

job = client.load_table_from_dataframe(
	customer_orders,
	destination_table,
	job_config=job_config
)

job.result()

LoadJob<project=explore29, location=US, id=5b306be9-db46-416b-bfb9-f18e9ebef380>

### Get ```runner_orders``` data

In [24]:
get_runner_data = """SELECT * FROM explore29.pizza_runner.runner_orders;"""
query = client.query(get_runner_data)
runner_orders = query.to_dataframe()

### Inspecting ```runner_orders```

In [25]:
display(
	runner_orders.head(),
	runner_orders.tail()
)

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,4,2,2020-01-04 13:53:03,23.4,40,None
1,5,3,2020-01-08 21:10:57,10,15,None
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,None
3,1,1,2020-01-01 18:15:34,20km,32 minutes,
4,2,1,2020-01-01 19:10:54,20km,27 minutes,


,order_id,runner_id,pickup_time,distance,duration,cancellation
5,9,2,null,null,null,Customer Cancellation
6,6,3,null,null,null,Restaurant Cancellation
7,10,1,2020-01-11 18:50:20,10km,10minutes,null
8,7,2,2020-01-08 21:30:45,25km,25mins,null
9,8,2,2020-01-10 00:15:02,23.4 km,15 minute,null


#### Clean missing/null values

In [26]:
runner_orders['pickup_time'] = runner_orders['pickup_time'].str.lower().str.strip().replace('', None).replace('null', None)
runner_orders['distance'] = runner_orders['distance'].str.lower().str.strip().replace('', None).replace('null', None)
runner_orders['duration'] = runner_orders['duration'].str.lower().str.strip().replace('', None).replace('null', None)
runner_orders['cancellation'] = runner_orders['cancellation'].str.lower().str.strip().replace('', None).replace('null', None).str.title()

runner_orders.tail()

,order_id,runner_id,pickup_time,distance,duration,cancellation
5,9,2,None,None,None,Customer Cancellation
6,6,3,None,None,None,Restaurant Cancellation
7,10,1,2020-01-11 18:50:20,10km,10minutes,None
8,7,2,2020-01-08 21:30:45,25km,25mins,None
9,8,2,2020-01-10 00:15:02,23.4 km,15 minute,None


### Convert data types

- ```pickup_time``` is converted to datatime
- ```distance``` is converted to float
- ```duration``` is converted to float

In [30]:
runner_orders['distance'] = runner_orders['distance'].apply(clean_numeric)
runner_orders['duration'] = runner_orders['duration'].apply(clean_numeric)
runner_orders['pickup_time'] = runner_orders['pickup_time'].apply(clean_datetime)

runner_orders.tail()

,order_id,runner_id,pickup_time,distance,duration,cancellation
5,9,2,NaT,NaN,NaN,Customer Cancellation
6,6,3,NaT,NaN,NaN,Restaurant Cancellation
7,10,1,2020-01-11 18:50:20,10.0,10.0,None
8,7,2,2020-01-08 21:30:45,25.0,25.0,None
9,8,2,2020-01-10 00:15:02,23.4,15.0,None


In [28]:
destination_table = 'explore29.pizza_runner.std_runner_orders'

job_config = bq.LoadJobConfig(
	write_disposition='WRITE_TRUNCATE'  # overwrite the table if it exists
)

job = client.load_table_from_dataframe(
	runner_orders,
	destination_table,
	job_config=job_config
)

job.result()

LoadJob<project=explore29, location=US, id=144be595-08ab-4e0c-a7d8-5c822515600a>